In [ ]:
import os

# Find the latest versions of
#   Spark & Hadoop:  https://www.apache.org/dist/spark/
#                    https://spark.apache.org/downloads.html
#   Postgres driver: https://jdbc.postgresql.org/
os.environ['HADOOP_VERSION']   = hadoop_version   = 'hadoop3'
os.environ['SPARK_VERSION']    = spark_version    = 'spark-3.3.1'
os.environ['POSTGRES_VERSION'] = postgres_version = 'postgresql-42.5.0'

# Install Java
! apt install openjdk-11-jdk-headless > /dev/null
os.environ['JAVA_HOME']  = '/usr/lib/jvm/java-11-openjdk-amd64'

# Install Spark
! wget https://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-$HADOOP_VERSION.tgz
! tar xf $SPARK_VERSION-bin-$HADOOP_VERSION.tgz
os.environ['SPARK_HOME'] = f'/content/{spark_version}-bin-{hadoop_version}'
! pip install findspark

# Install Postgres driver
! wget https://jdbc.postgresql.org/download/$POSTGRES_VERSION.jar

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName('AmazonReviews') \
  .config('spark.driver.extraClassPath', f'/content/{postgres_version}.jar') \
  .getOrCreate()
  
spark



--2022-11-07 13:35:50--  https://www.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving www.apache.org (www.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to www.apache.org (www.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz [following]
--2022-11-07 13:35:50--  https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz.1’

spark-3.3.1-bin-had 100%[===================>] 285.48M  28.3MB/s    in 11s     

2022-11-07 13:36:01 (26.3 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz.1’ saved [2993

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
# Count the number of records (rows) in the dataset.
df.count()

1785997

In [ ]:
# Transform the dataset to fit the tables in PostgreSQL.
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



customer_id to be changed to INT

helpful_votes to be changed to INT

product_parent to be chenged to INT

review_date to be changed to date format (yyyy-mm-dd)

star_rating to be chenged to INT

total_votes to be changed to INT

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType

In [ ]:
schema = [StructField("marketplace", StringType(), True), StructField("customer_id", IntegerType(), True),StructField("review_id", StringType(), True),StructField("product_id", StringType(), True),StructField("product_parent", IntegerType(), True),StructField("product_title", StringType(), True),StructField("product_category", StringType(), True),StructField("star_rating", IntegerType(), True),StructField("helpful_votes", IntegerType(), True),StructField("total_votes", IntegerType(), True),StructField("vine", StringType(), True),StructField("verified_purchase", StringType(), True),StructField("review_headline", StringType(), True),StructField("review_body", StringType(), True),StructField("review_date", DateType(), True),]
schema

[StructField('marketplace', StringType(), True),
 StructField('customer_id', IntegerType(), True),
 StructField('review_id', StringType(), True),
 StructField('product_id', StringType(), True),
 StructField('product_parent', IntegerType(), True),
 StructField('product_title', StringType(), True),
 StructField('product_category', StringType(), True),
 StructField('star_rating', IntegerType(), True),
 StructField('helpful_votes', IntegerType(), True),
 StructField('total_votes', IntegerType(), True),
 StructField('vine', StringType(), True),
 StructField('verified_purchase', StringType(), True),
 StructField('review_headline', StringType(), True),
 StructField('review_body', StringType(), True),
 StructField('review_date', DateType(), True)]

In [ ]:
final = StructType(fields=schema)
final

StructType([StructField('marketplace', StringType(), True), StructField('customer_id', IntegerType(), True), StructField('review_id', StringType(), True), StructField('product_id', StringType(), True), StructField('product_parent', IntegerType(), True), StructField('product_title', StringType(), True), StructField('product_category', StringType(), True), StructField('star_rating', IntegerType(), True), StructField('helpful_votes', IntegerType(), True), StructField('total_votes', IntegerType(), True), StructField('vine', StringType(), True), StructField('verified_purchase', StringType(), True), StructField('review_headline', StringType(), True), StructField('review_body', StringType(), True), StructField('review_date', DateType(), True)])

In [ ]:
dataframe = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), schema=final, sep="\t", header=True)
dataframe.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
#Table1 is review_id, customer_id, product_id, product_parent, review_date
review_id_table = dataframe.select(['review_id','customer_id','product_id','product_parent','review_date'])

review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

In [ ]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
#Table2 is product_id, product_title
products = dataframe.select(['product_id','product_title'])

products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [ ]:
products.count()

1785997

In [ ]:
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CXYMFS|Thrustmaster T-Fl...|
|B00M920ND6|Tonsee 6 buttons ...|
|B0029CSOD2|Hidden Mysteries:...|
|B00GOOSV98|GelTabz Performan...|
|B00Y074JOM|Zero Suit Samus a...|
|B002UBI6W6|Psyclone Recharge...|
|B008XHCLFO|Protection for yo...|
|B00BRA9R6A|   Nerf 3DS XL Armor|
|B009EPWJLA|One Piece: Pirate...|
|B0000AV7GB|Playstation 2 Dan...|
|B00008KTNN|Metal Arms: Glitc...|
|B000A3IA0Y|72 Pin Connector ...|
|B0053BQN34|uDraw Gametablet ...|
|B002I0HIMI|NBA 2K12(Covers M...|
|B0081EH12M|New Trigger Grips...|
|B005FMLZQQ|Xbox 360 Media Re...|
|B002BSA388|Super Mario Galaxy 2|
|B00BUSLSAC|Nintendo 3DS XL -...|
|B00KWF38AW|Captain Toad:  Tr...|
|B00BRQJYA8|Lego Batman 2: DC...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
products_check = products.groupBy('product_id').count()
products_check.orderBy("count", ascending=False).select(["product_id", "count"])

products_check.show()

+----------+-----+
|product_id|count|
+----------+-----+
|B00TS0UKE4|  119|
|B004LQRF3Y|  142|
|B002I0K3Z2|  518|
|B002ZKTHD4|   83|
|B00BMFIXT2| 1467|
|B00XMQBA62|   18|
|B00N4ABMG4|  136|
|B00IEYG1AC|   29|
|B00TNY43ZC|    1|
|B006VB2UNM| 1051|
|B00HPU96FI|   76|
|B00HSY1R3K|   14|
|B002BZ11E6|   39|
|B000YQ32TG|   39|
|B002EE1P2W|  159|
|B006WQR3GA|  311|
|B00005MDZ8|  155|
|B009RBVXR2|    1|
|B00VJSCNXO|    1|
|B00002STA7|    6|
+----------+-----+
only showing top 20 rows



In [ ]:
products = products.dropDuplicates()

products.count()

65792

In [ ]:
products_check = products.groupBy('product_id').count()
products_check.orderBy("count", ascending=False).select(["product_id", "count"])

products_check.show()

+----------+-----+
|product_id|count|
+----------+-----+
|B0006GT76G|    1|
|B008BDZ3RI|    1|
|B0014KT4SO|    1|
|B0002BJQXO|    1|
|B003V8Q7LE|    1|
|B0048CAW54|    1|
|B000067AVJ|    1|
|B00337KBUQ|    1|
|B000V5HCUA|    1|
|B0000296ZL|    1|
|B00004T9BT|    1|
|B00002SW49|    1|
|B000VM3RRK|    1|
|B002ZKTHD4|    1|
|B00DBCAS7E|    1|
|B000FNQXQW|    1|
|B000069TEX|    1|
|B0018IKIEQ|    1|
|B006Q6PYUS|    1|
|B005NROW5E|    1|
+----------+-----+
only showing top 20 rows



In [ ]:
products = products.dropna()

products.count()

65792

In [ ]:
products = products.orderBy('product_id')

products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0000118532|Bethesda The Evil...|
|006056038X|        Bitter Truth|
|006073132X|Freakonomics: A R...|
|007876355X|Skylanders Giants...|
|0078764343|Medal of Honor: W...|
|0133617300|Disney Infinity F...|
|0182116247|LCD Screen Displa...|
|0182116255|2-Pack Black Plas...|
|0182116263|3D Analog Joystic...|
|0226739368|Into the Cool: En...|
|0324411812|  Xbox 360 MAS STICK|
|0385510365|Don't Get Too Com...|
|0439026008|Oregon Trail 5th ...|
|043933702X|Need for Speed Po...|
|0439339987|Freddi Fish and t...|
|0439394422|school house rock...|
|0439573947|Need for Speed Ra...|
|0439574447|Thomas & Friends ...|
|0439591295|Scooby-do Mystery...|
|0439591511|Zapper- One Wicke...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
#Table3 is customer_id, customer_count - is this count of reviews?

customers = dataframe.select(['customer_id'])

customers.show()

+-----------+
|customer_id|
+-----------+
|   12039526|
|    9636577|
|    2331478|
|   52495923|
|   14533949|
|    2377552|
|   17521011|
|   19676307|
|     224068|
|   48467989|
|     106569|
|   48269642|
|   52738710|
|   10556786|
|    2963837|
|   23092109|
|   23091728|
|   10712640|
|   17455376|
|   14754850|
+-----------+
only showing top 20 rows



In [ ]:
customers = customers.groupBy('customer_id').count()
customers.orderBy("customer_id").select(["customer_id", "count"])


customers.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   48670265|    1|
|   49103216|    2|
|    1131200|    1|
|   43076447|    2|
|   46261368|    1|
|    4883305|    5|
|   41192649|    1|
|   40985731|    7|
|   10437900|    2|
|   22245671|    1|
|    2574873|    1|
|    4696154|    1|
|    5621202|    1|
|    5871933|    2|
|   44089812|    1|
|    2845910|    1|
|    5274369|    1|
|   39069693|    2|
|     137793|    1|
|   31914942|    3|
+-----------+-----+
only showing top 20 rows



In [ ]:
customers = customers.withColumnRenamed('count','customer_count')

customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customers_df = customers.withColumn("customer_id",customers["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customers["customer_count"].cast(IntegerType()))

customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [ ]:
#Table4 is review_id, star_rating, helpful_votes, total_votes, vine
vine_table = dataframe.select(['review_id','star_rating','helpful_votes','total_votes','vine'])

vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/<database_name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table in RDS

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [27]:
customers.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [28]:
vine_table.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)